# [HW5] FBA QUANT - FINANCIAL ENGINEERING

Kim Na Young (dudskrla09@gmail.com)

---

### Problem 1. 


Reproduce the payer-swaption lattices in the Coursera Lecture. Now experiment with different values of b (but re-calibrating each time so that the spot rate curve remains unchanged) to see how sensitive the swaption price is to the particular value of b that you choose. (Among other things, this should highlight the importance of calibrating models correctly and understanding whether or not your model is appropriate for the problem at hand. For example, we know that option prices are sensitive to volatility and since b is a volatility parameter in the BDT model, it is clearly important to calibrate it accurately. In fact we may wish to choose and  calibrate a separate bi for each time period. If we were pricing a swaption where the underlying swap expired after 10 years instead of just 10 months we would also want to consider whether or not a model that cannot, for example, incorporate mean-reversion should be used for such a task.)


### Solution 1.

[Assumption]
- (1) **The notional principal** is set $1
- (2) **The fixed rate** is set at 11.65%
- (3) **The spot rate in market**         

    | $s_1$ | $s_2$ | $s_3$ | $s_4$ | $s_5$ | $s_6$ | $s_7$ | $s_8$ | $s_9$ | $s_{10}$ |
    |-------|-------|-------|-------|-------|-------|-------|-------|-------|---------|
    | 7.3   | 7.62  | 8.1   | 8.45  | 9.2   | 9.64  | 10.12 | 10.45 | 10.75 | 11.22   |

[Sol]
- **Step 1. Find the short rate** 
$$ short \; rate \; (r_{i, \; j}) = a_ie^{b_i*j} $$
- (1) **Find the $ b_i $**           
    We assumed that $ b_i $ is fixed (ex. $ b_i = 0.005 $)
- (2) **Find the $ a_i $**    
    Using following equation, we can find $ a_i $
    $$ (1) \; \frac{1}{(1+s_i)^i} = \sum_{j=0}^{i}P^e_{i, j} $$
    $$ = \frac{P^e_{i-1, 0}}{2(1+a_{i-1})} + \sum_{j=1}^{i-1} (\frac{P^e_{i-1, j}}{2(1+a_{i-1}e^{bj})} + \frac{P^e_{i-1, j-1}}{2(1+a_{i-1}e^{b(j-1)})}) + \frac{P^e_{i-1, j-1}}{2(1+a_{i-1}e^{b(i-1)})} $$

    To find the $ a_i $, find elementary price $ P^e $ first
- (3) **Find the elementary price $ P^e $**      
    Using **forward equations**, we can find $ P^e $

    $$P_e(k + 1, s) = \frac{1}{2}\frac{P_e(k, s - 1)}{(1 + r_{k,s-1})} + \frac{1}{2}\frac{P_e(k, s)}{(1 + r_{k,s})}, \quad 0 < s < k + 1$$

    $$P_e(k + 1, 0) = \frac{1}{2} \frac{P_e(k, 0)}{(1 + r_{k,0})}$$

    $$P_e(k + 1, k + 1) = \frac{1}{2} \frac{P_e(k, k)}{(1 + r_{k,k})}$$

---

### 1) Initialize a = 5 at all i

In [1]:
import numpy as np
import pandas as pd

# Initialize variables 
P_0 = 1 # notional principle 
s = [7.3, 7.62, 8.1, 8.45, 9.2, 9.64,  10.12, 10.45, 10.75, 11.22] # spot rate of market
c = 11.65 / 100 # fixed rate 

T = 10 # swap maturity
E = 2 # option expiration
a = [5] * (T+1)
b = [0.005] * (T+1)

- **Step 1. Find the short rate** 
$$ short \; rate \; (r_{i, \; j}) = a_ie^{b_i*j} $$

In [2]:
def find_short_rate(T, a, b):
    df_short_rate = pd.DataFrame(np.zeros((T)*(T)).reshape(T, T))
    df_short_rate.index = [idx for idx in range(T-1, -1, -1)]
    for col in range(0, T):
        for row in range(0, col+1):
            df_short_rate.loc[row][col] = a[col] * np.exp(b[col] * row) / 100 # (%)
    return df_short_rate

- **Step 2. Find the elementary price $ P^e $**      
    Using **forward equations**, we can find $ P^e $

    $$P_e(k + 1, s) = \frac{1}{2}\frac{P_e(k, s - 1)}{(1 + r_{k,s-1})} + \frac{1}{2}\frac{P_e(k, s)}{(1 + r_{k,s})}, \quad 0 < s < k + 1$$

    $$P_e(k + 1, 0) = \frac{1}{2} \frac{P_e(k, 0)}{(1 + r_{k,0})}$$

    $$P_e(k + 1, k + 1) = \frac{1}{2} \frac{P_e(k, k)}{(1 + r_{k,k})}$$

After finding the elementary price, we will use this value to find the spot rate of model.
    $$ \frac{1}{(1+s_i)^i} = \sum_{j=0}^{i}P^e_{i, j} $$
    $$ = \frac{P^e_{i-1, 0}}{2(1+a_{i-1})} + \sum_{j=1}^{i-1} (\frac{P^e_{i-1, j}}{2(1+a_{i-1}e^{bj})} + \frac{P^e_{i-1, j-1}}{2(1+a_{i-1}e^{b(j-1)})}) + \frac{P^e_{i-1, j-1}}{2(1+a_{i-1}e^{b(i-1)})} $$

In [3]:
def find_elementary_price(T, P_0, df_short_rate):
    df_elementary_price = pd.DataFrame(np.zeros((T)*(T)).reshape(T, T))
    df_elementary_price.index = [idx for idx in range(T-1, -1, -1)]
    df_elementary_price[0][0] = P_0 
    
    for col in range(1, T):
        for row in range(0, col+1):
            # (1) j == 0
            if row == 0:
                df_elementary_price.loc[row][col] = (1/2) * (df_elementary_price.loc[row][col-1]) / (1+df_short_rate.loc[row][col-1])
         
            # (2) i == j 
            elif row == col:
                df_elementary_price.loc[row][col] = (1/2) * (df_elementary_price.loc[row-1][col-1]) / (1+df_short_rate.loc[row-1][col-1])
            
            # (3) i > j
            elif row < col:
                df_elementary_price.loc[row][col] = (1/2) * (df_elementary_price.loc[row-1][col-1]) / (1+df_short_rate.loc[row-1][col-1]) \
                                                    + (1/2) * (df_elementary_price.loc[row][col-1]) / (1+df_short_rate.loc[row][col-1])
            
    return df_elementary_price

In [4]:
def find_swaption_price(T, df_short_rate, c):
    df_swaption_price = pd.DataFrame(np.zeros((T)*(T)).reshape(T, T))
    df_swaption_price.index = [idx for idx in range(T-1, -1, -1)]
    
    # (1) t = Swap maturity (t=10)
    for row in range(0, T):
        df_swaption_price.loc[row][T-1] = (df_short_rate.loc[row][T-1] - c) / (1 + df_short_rate.loc[row][T-1])
        
    # (2) Option expiration (t=2) < t < Swap maturity (t=10)
    for col in range((T-1)-1, E, -1):
        for row in range(0, col+1):
            df_swaption_price.loc[row][col] = ( df_short_rate.loc[row][col] - c \
                                                + (1/2) * df_swaption_price.loc[row][col+1] + (1/2) * df_swaption_price.loc[row+1][col+1] ) \
                                                / (1 + df_short_rate.loc[row][col])
    # (3) t = Option expiration (t=2)
    for row in range(0, E+1):
        df_swaption_price.loc[row][E] = max(0, \
                                        ( df_short_rate.loc[row][E] - c \
                                                + (1/2) * df_swaption_price.loc[row][E+1] + (1/2) * df_swaption_price.loc[row+1][E+1] ) \
                                                / (1 + df_short_rate.loc[row][E])
                                        )

    # (4) t < Option expiration (t=2) 
    for col in range(E-1, -1, -1):
        for row in range(0, col+1):
            df_swaption_price.loc[row][col] = ((1/2) * df_swaption_price.loc[row][col+1] + (1/2) * df_swaption_price.loc[row+1][col+1])\
                                            / (1 + df_short_rate.loc[row][col])
    
    return df_swaption_price

In [5]:

df_short_rate = find_short_rate(T+1, a, b)
df_elementary_price = find_elementary_price(T+1, P_0, df_short_rate) 
df_swaption_price = find_swaption_price(T, df_short_rate, c)

In [6]:
print(f"{E}-{T-E} payer swaption price (a=5, b=0.005 at all i)")
df_swaption_price

2-8 payer swaption price (a=5, b=0.005 at all i)


,0,1,2,3,4,5,6,7,8,9
9,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.061008
8,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.119386,-0.061271
7,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.175291,-0.119912,-0.061533
6,0.0,0.0,0.0,0.000000,0.000000,0.000000,-0.228870,-0.176079,-0.120435,-0.061794
5,0.0,0.0,0.0,0.000000,0.000000,-0.280261,-0.229918,-0.176863,-0.120957,-0.062053
4,0.0,0.0,0.0,0.000000,-0.329591,-0.281567,-0.230962,-0.177644,-0.121476,-0.062311
3,0.0,0.0,0.0,-0.376983,-0.331155,-0.282869,-0.232002,-0.178423,-0.121993,-0.062569
2,0.0,0.0,0.0,-0.378801,-0.332713,-0.284167,-0.233038,-0.179197,-0.122507,-0.062825
1,0.0,0.0,0.0,-0.380612,-0.334265,-0.285459,-0.234071,-0.179969,-0.123020,-0.063080
0,0.0,0.0,0.0,-0.382418,-0.335813,-0.286747,-0.235099,-0.180738,-0.123530,-0.063333


- **Step 3. find the BDT Model ZCB Prices**

After finding the elementary price, we will use this value to find the spot rate of model.
    $$ \frac{1}{(1+s_i)^i} = \sum_{j=0}^{i}P^e_{i, j} $$
    $$ = \frac{P^e_{i-1, 0}}{2(1+a_{i-1})} + \sum_{j=1}^{i-1} (\frac{P^e_{i-1, j}}{2(1+a_{i-1}e^{bj})} + \frac{P^e_{i-1, j-1}}{2(1+a_{i-1}e^{b(j-1)})}) + \frac{P^e_{i-1, j-1}}{2(1+a_{i-1}e^{b(i-1)})} $$

In [7]:
print(f"Elementary price (a=5, b=0.005 at all i)")
df_elementary_price

Elementary price (a=5, b=0.005 at all i)


,0,1,2,3,4,5,6,7,8,9,10
10,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000593
9,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001248,0.005937
8,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002626,0.011244,0.026746
7,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005524,0.021027,0.045020,0.071400
6,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.011618,0.038698,0.073657,0.105148,0.125086
5,0.0,0.00000,0.000000,0.000000,0.000000,0.024427,0.069748,0.116177,0.147438,0.157875,0.150267
4,0.0,0.00000,0.000000,0.000000,0.051345,0.122192,0.174476,0.193769,0.184453,0.158027,0.125358
3,0.0,0.00000,0.000000,0.107902,0.205454,0.244501,0.232774,0.193909,0.147687,0.105453,0.071711
2,0.0,0.00000,0.226703,0.323784,0.308292,0.244618,0.174685,0.116429,0.073906,0.045238,0.026921
1,0.0,0.47619,0.453461,0.323862,0.205602,0.122368,0.069916,0.038838,0.021134,0.011320,0.005989


In [8]:
bdt_zcb_price = [0] * (T+1)

for col in range(1, T+1):
    bdt_zcb_price[col] = sum(df_elementary_price.loc[:, col])

df_bdt_zcb_price = pd.DataFrame({"BDT Model ZCB Prices" : bdt_zcb_price}).T

print(f"BDT Model ZCB Prices: ")
df_bdt_zcb_price

BDT Model ZCB Prices: 


,0,1,2,3,4,5,6,7,8,9,10
BDT Model ZCB Prices,0.0,0.952381,0.906921,0.863528,0.822113,0.78259,0.744877,0.708896,0.674572,0.641831,0.610606


- **Step 4. BDT Model Spot Rates**

Using following equation, we can find $ s_i $
$$ \frac{1}{(1+s_i)^i} = \sum_{j=0}^{i}P^e_{i, j} $$
$$ ∴ \; s_i = \frac{1}{\sum_{j=0}^{i}P^e_{i, j}} ^ {\frac{1}{i}} - 1 = \frac{1}{BDT \; Model \; ZCB \; Prices} ^ {\frac{1}{i}} - 1$$

In [9]:
bdt_spot_rate = [0] * (T+1)

for col in range(1, T+1):
    bdt_spot_rate[col] = ( (1 / df_bdt_zcb_price.iloc[0][col]) ** (1 / col) - 1 ) * 100 # (%)

df_bdt_spot_rate= pd.DataFrame({"BDT Model Spot Rates" : bdt_spot_rate[1:]}).T

print(f"BDT Model Spot Rates: ")
df_bdt_spot_rate

BDT Model Spot Rates: 


,0,1,2,3,4,5,6,7,8,9
BDT Model Spot Rates,5.0,5.006265,5.012539,5.018824,5.025118,5.031422,5.037736,5.04406,5.050394,5.056738


In [10]:
df_market_spot_rate = pd.DataFrame({"Market Spot Rates" : s}).T
df_market_spot_rate

,0,1,2,3,4,5,6,7,8,9
Market Spot Rates,7.3,7.62,8.1,8.45,9.2,9.64,10.12,10.45,10.75,11.22


### 2) Optimize a value to minimize the difference of spot rate between BDT model and market

- Objective) spot rate of BDT model ≒ spot rate of market

In [11]:
import numpy as np
import pandas as pd

# Initialize variables 
P_0 = 1 # notional principle 
s = [7.3, 7.62, 8.1, 8.45, 9.2, 9.64,  10.12, 10.45, 10.75, 11.22] # spot rate of market
c = 11.65 / 100 # fixed rate 

T = 10 # swap maturity
E = 2 # option expiration

- Change the value of b

In [12]:
list_b = [0.001, 0.005, 0.010, 0.050, 0.100, 0.500]
list_b = [[item] * (T+1) for item in list_b] # CHANGE THIS VALUE 

In [13]:
from tqdm import tqdm
from scipy.optimize import minimize

dict_a = dict()

for b in tqdm(list_b):
    # Define the objective function to minimize
    def objective_function(a):
        # Step 1. find the short rate 
        df_short_rate = find_short_rate(T+1, a, b)
        
        # Step 2. find the elementary price 
        df_elementary_price = find_elementary_price(T+1, P_0, df_short_rate) 
        
        # Step 3. find the zcb price 
        bdt_zcb_price = [0] * (T+1)
        for col in range(1, T+1):
            bdt_zcb_price[col] = sum(df_elementary_price.loc[:, col])
        
        # Step 4. find the spot rate in model 
        bdt_spot_rate = [0] * (T+1)
        for col in range(1, T+1):
            bdt_spot_rate[col] = ( (1 / bdt_zcb_price[col]) ** (1 / col) - 1 ) * 100 # (%)
        bdt_spot_rate = bdt_spot_rate[1:]
        
        # Step 5. minimize the difference of spot rate between model and market
        squared_difference = [(bdt_spot_rate[i] - s[i])**2 for i in range(T)]
        
        return sum(squared_difference)

    initial_a = [5.0] * (T+1)

    # Minimize the objective function to find the optimal a_i values
    result = minimize(objective_function, initial_a, method='BFGS')

    # Extract the optimal a_i values
    optimal_a = result.x
    dict_a[f"a (where b={b[0]})"] = optimal_a[:-1]

dict_a["Spot rate in market"] = s
df_optimal_a = pd.DataFrame(dict_a).T
df_optimal_a


  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [01:28<00:00, 14.70s/it]


,0,1,2,3,4,5,6,7,8,9
a (where b=0.001),7.299999,7.936985,9.057381,9.492545,12.227742,11.837153,13.005265,12.743302,13.126804,15.471057
a (where b=0.005),7.300000,7.921107,9.021166,9.435759,12.130085,11.719584,12.849712,12.566379,12.918267,15.195255
a (where b=0.01),7.299999,7.901266,8.976058,9.365038,12.009254,11.573578,12.658457,12.348053,12.662564,14.857252
a (where b=0.05),7.300000,7.742829,8.620155,8.814408,11.078815,10.465783,11.221380,10.731323,10.789647,12.414277
a (where b=0.1),7.300000,7.545623,8.188073,8.162194,10.004652,9.218109,9.643356,8.999305,8.832150,9.924301
a (where b=0.5),7.300003,6.022740,5.246092,4.222974,4.227090,3.200470,2.779692,2.167126,1.791134,1.721088
Spot rate in market,7.300000,7.620000,8.100000,8.450000,9.200000,9.640000,10.120000,10.450000,10.750000,11.220000


- **Step 5. Find the swaption price**

- Short rate (After optimizing)

In [14]:
dict_short_rate = dict()

for idx, b in tqdm(enumerate(list_b)):
    a = df_optimal_a.iloc[idx]
    
    def find_short_rate(T, a, b):
        df_short_rate = pd.DataFrame(np.zeros((T)*(T)).reshape(T, T))
        df_short_rate.index = [idx for idx in range(T-1, -1, -1)]
        for col in range(0, T):
            for row in range(0, col+1):
                df_short_rate.loc[row][col] = a[col] * np.exp(b[col] * row) / 100 # (%)
        return df_short_rate

    df_optimal_short_rate = find_short_rate(T, a, b)
    dict_short_rate[f"short rate (where b={b[0]})"] = df_optimal_short_rate

6it [00:00, 113.64it/s]


- Elementary price (After optimizing)

In [15]:
dict_elementary_price = dict()

for idx, b in tqdm(enumerate(list_b)):
    df_short_rate = dict_short_rate[f"short rate (where b={b[0]})"]
    
    def find_elementary_price(T, P_0, df_short_rate):
        df_elementary_price = pd.DataFrame(np.zeros((T)*(T)).reshape(T, T))
        df_elementary_price.index = [idx for idx in range(T-1, -1, -1)]
        df_elementary_price[0][0] = P_0 
        
        for col in range(1, T):
            for row in range(0, col+1):
                # (1) j == 0
                if row == 0:
                    df_elementary_price.loc[row][col] = (1/2) * (df_elementary_price.loc[row][col-1]) / (1+df_short_rate.loc[row][col-1])
            
                # (2) i == j 
                elif row == col:
                    df_elementary_price.loc[row][col] = (1/2) * (df_elementary_price.loc[row-1][col-1]) / (1+df_short_rate.loc[row-1][col-1])
                
                # (3) i > j
                elif row < col:
                    df_elementary_price.loc[row][col] = (1/2) * (df_elementary_price.loc[row-1][col-1]) / (1+df_short_rate.loc[row-1][col-1]) \
                                                        + (1/2) * (df_elementary_price.loc[row][col-1]) / (1+df_short_rate.loc[row][col-1])
        return df_elementary_price

    df_optimal_elementary_price = find_elementary_price(T, P_0, df_short_rate) 
    dict_elementary_price[f"elementary price (where b={b[0]})"] = df_optimal_elementary_price

6it [00:00, 40.62it/s]


- Swap price (After optimizing)

In [16]:
dict_swap_price = dict()

for idx, b in tqdm(enumerate(list_b)):
    df_short_rate = dict_short_rate[f"short rate (where b={b[0]})"]
    
    def find_swaption_price(T, df_short_rate, c):
        df_swaption_price = pd.DataFrame(np.zeros((T)*(T)).reshape(T, T))
        df_swaption_price.index = [idx for idx in range(T-1, -1, -1)]
        
        # (1) t = Swap maturity (t=10)
        for row in range(0, T):
            df_swaption_price.loc[row][T-1] = (df_short_rate.loc[row][T-1] - c) / (1 + df_short_rate.loc[row][T-1])
            
        # (2) Option expiration (t=2) < t < Swap maturity (t=10)
        for col in range((T-1)-1, E, -1):
            for row in range(0, col+1):
                df_swaption_price.loc[row][col] = ( df_short_rate.loc[row][col] - c \
                                                    + (1/2) * df_swaption_price.loc[row][col+1] + (1/2) * df_swaption_price.loc[row+1][col+1] ) \
                                                    / (1 + df_short_rate.loc[row][col])
        # (3) t = Option expiration (t=2)
        for row in range(0, E+1):
            df_swaption_price.loc[row][E] = max(0, \
                                            ( df_short_rate.loc[row][E] - c \
                                                    + (1/2) * df_swaption_price.loc[row][E+1] + (1/2) * df_swaption_price.loc[row+1][E+1] ) \
                                                    / (1 + df_short_rate.loc[row][E])
                                            )

        # (4) t < Option expiration (t=2) 
        for col in range(E-1, -1, -1):
            for row in range(0, col+1):
                df_swaption_price.loc[row][col] = ((1/2) * df_swaption_price.loc[row][col+1] + (1/2) * df_swaption_price.loc[row+1][col+1])\
                                                / (1 + df_short_rate.loc[row][col])
        return df_swaption_price

    df_optimal_swaption_price = find_swaption_price(T, df_short_rate, c)
    dict_swap_price[f"swap price (where b={b[0]})"] = df_optimal_swaption_price

0it [00:00, ?it/s]

6it [00:00, 41.55it/s]


In [17]:
dict_result = dict()
for idx, b in tqdm(enumerate(list_b)):
    swap_price = round(dict_swap_price[f"swap price (where b={b[0]})"][0][0], 5)
    dict_result[f"where b={b[0]}"] = swap_price
df_swap_price = pd.DataFrame(dict_result, index=["Swap price"]).T
df_swap_price

6it [00:00, ?it/s]


,Swap price
where b=0.001,0.00096
where b=0.005,0.00134
where b=0.01,0.00196
where b=0.05,0.00693
where b=0.1,0.01310
where b=0.5,0.05771


∴ b ↑ ∝ Swap price ↑

(∵ b is a volatility parameter for short rate. If the volatility of short rate is increased, then the swaption price is increased.)

----


### Problem 2. 

Price the payer-swaption in the Coursera Lecture but now assume that it may be exercised at any time, t ∈ {2,3, ⋯ ,9}, and that the fixed rate in the underlying swap contract is now set at 11.65%. If exercised at time t then the first cash flow occurs at t + 1 based on the short rate prevailing at time t. (Such an instrument is called a Bermudan swaption.)


### Solution 2.

[Assumption]
- (1) **The notional principal** is set $1
- (2) **The fixed rate** is set at 11.65%
- (3) **The spot rate in market**         

    | $s_1$ | $s_2$ | $s_3$ | $s_4$ | $s_5$ | $s_6$ | $s_7$ | $s_8$ | $s_9$ | $s_{10}$ |
    |-------|-------|-------|-------|-------|-------|-------|-------|-------|---------|
    | 7.3   | 7.62  | 8.1   | 8.45  | 9.2   | 9.64  | 10.12 | 10.45 | 10.75 | 11.22   |
- (4) **The volatility parameter (b)** is 0.005 at all i

In [18]:
dict_swap_price[f"swap price (where b={b[0]})"] = df_optimal_swaption_price

In [19]:
b = 0.005
df_short_rate = dict_short_rate[f"short rate (where b={b})"]
df_swap_price = dict_swap_price[f"swap price (where b={b})"]

In [20]:
def find_swaption_price(T, df_short_rate, c):
    early_exercise_date = E
    df_bermudan_swap_price = pd.DataFrame(np.zeros((T)*(T)).reshape(T, T))
    df_bermudan_swap_price.index = [idx for idx in range(T-1, -1, -1)]
    
    # (1) t = Swap maturity (t=10)
    for row in range(0, T):
        df_bermudan_swap_price.loc[row][T-1] = (df_short_rate.loc[row][T-1] - c) / (1 + df_short_rate.loc[row][T-1])
        
    # (2) t < Swap maturity (t=10)
    for col in range((T-1)-1, -1, -1):        
        for row in range(0, col+1):
            left = df_swap_price.loc[row][col]
            right = ((1/2) * df_bermudan_swap_price.loc[row][col+1] + (1/2) * df_bermudan_swap_price.loc[row+1][col+1])\
                    / (1 + df_short_rate.loc[row][col])
                    
            if left > right : # early exercise
                df_bermudan_swap_price.loc[row][col] = left
            else:
                df_bermudan_swap_price.loc[row][col] = right
    
    return df_bermudan_swap_price

df_bermudan_swap_price = find_swaption_price(T, df_short_rate, c)

print(f"{E}-{T-E} Bermudan swaption price (a=5, b=0.005 at all i) : {round(df_bermudan_swap_price[0][0], 5)}")
df_bermudan_swap_price

2-8 Bermudan swaption price (a=5, b=0.005 at all i) : 0.0369


,0,1,2,3,4,5,6,7,8,9
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036625
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047820,0.035966
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.053877,0.046677,0.035309
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060941,0.052322,0.045537,0.034654
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.056816,0.059021,0.050772,0.044400,0.034002
4,0.000000,0.000000,0.000000,0.000000,0.056025,0.054598,0.057105,0.049225,0.043267,0.033352
3,0.000000,0.000000,0.000000,0.049991,0.053534,0.052384,0.055194,0.047683,0.042137,0.032704
2,0.000000,0.000000,0.044785,0.047740,0.051045,0.050174,0.053286,0.046144,0.041012,0.032059
1,0.000000,0.040535,0.042739,0.045489,0.048560,0.047968,0.051383,0.044610,0.039889,0.031416
0,0.036901,0.038654,0.040693,0.043239,0.046078,0.045766,0.049484,0.043080,0.038770,0.030776


---

### Problem 3.

Construct a short lattice for periods (years) 0 through 9 with an initial rate of 6% and with successive rates determined by a multiplicative factor of either u = 1.2 or d = .9. Assign the risk-neutral probabilities to be .5. 

(a) Using this lattice, find the value of a 10-year 6% bond.           
(b) Suppose this bond can be called by the issuing party at any time after 5 years. (When the bond is called, the face value plus the currently due coupon are paid at that time and the bond is canceled.) What is the fair value of this bond?          
(c) Use the forward equation to find the spot rate curve for the lattice. 

### Solution 3.

- (a) Using this lattice, find the value of a 10-year 6% bond. 

In [26]:
import numpy as np
import pandas as pd

t = 10
face_value = 100
initial_rate = 0.06
u = 1.2
d = 0.9
coupon = face_value * initial_rate

In [27]:
# Initialize Short rate 
df_short_rate = pd.DataFrame(np.zeros((t)*(t)).reshape(t,t))
df_short_rate.index = [idx for idx in range(t-1, -1, -1)]

# Initialize iloc[-1] 
df_short_rate.loc[0][0] = initial_rate

for col in range(1, t):
    df_short_rate.loc[0][col] = df_short_rate.loc[0][col-1] * d # down : 0.9 

for row in range(1, t):
    for col in range(row, t):
        df_short_rate.loc[row][col]  = df_short_rate.loc[row-1][col-1] * u # up : 1.2

print("Short Rate: ")
df_short_rate

Short Rate: 


,0,1,2,3,4,5,6,7,8,9
9,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.309587
8,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.000000,0.000000,0.257989,0.232190
7,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.000000,0.214991,0.193492,0.174143
6,0.00,0.000,0.0000,0.00000,0.000000,0.000000,0.179159,0.161243,0.145119,0.130607
5,0.00,0.000,0.0000,0.00000,0.000000,0.149299,0.134369,0.120932,0.108839,0.097955
4,0.00,0.000,0.0000,0.00000,0.124416,0.111974,0.100777,0.090699,0.081629,0.073466
3,0.00,0.000,0.0000,0.10368,0.093312,0.083981,0.075583,0.068024,0.061222,0.055100
2,0.00,0.000,0.0864,0.07776,0.069984,0.062986,0.056687,0.051018,0.045917,0.041325
1,0.00,0.072,0.0648,0.05832,0.052488,0.047239,0.042515,0.038264,0.034437,0.030994
0,0.06,0.054,0.0486,0.04374,0.039366,0.035429,0.031886,0.028698,0.025828,0.023245


In [28]:
# Initialize Bond price
df_bond_price = pd.DataFrame(np.zeros((t+1)*(t+1)).reshape(t+1,t+1))
df_bond_price.index = [idx for idx in range(t, -1, -1)]

# Initialize t=4 as 100 (Face value) 
for row in range(0, t+1):
    df_bond_price.loc[row][t] = face_value

for col in range(t-1, -1, -1):
    for row in range(0, col+1):
        short_rate = df_short_rate.loc[row][col]
        upper_node = df_bond_price.loc[row+1][col+1]
        lower_node = df_bond_price.loc[row][col+1]
        df_bond_price.loc[row][col]  = ( (1/2) * upper_node + (1/2) * lower_node) / (1 + short_rate)

print("The price of a 10-period zero-coupon bond: ")
df_bond_price

The price of a 10-period zero-coupon bond: 


,0,1,2,3,4,5,6,7,8,9,10
10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,76.359962,100.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,62.606377,81.156308,100.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,54.439219,69.679928,85.168532,100.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,49.646430,62.642855,75.807242,88.448068,100.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,47.019220,58.431875,69.923793,80.952441,91.078397,100.0
4,0.000000,0.000000,0.000000,0.000000,45.858476,56.108787,66.351195,76.151940,85.165289,93.156152,100.0
3,0.000000,0.000000,0.000000,45.736744,55.098985,64.371976,73.204777,81.323646,88.545995,94.777764,100.0
2,0.000000,0.000000,46.377471,55.032225,63.524077,71.567517,78.945702,85.518156,91.216304,96.031512,100.0
1,0.000000,47.591014,55.657662,63.496333,70.874800,77.622237,83.632396,88.857946,93.299665,96.993809,100.0


In [29]:
# Initialize Bond price
df_forward_contract = pd.DataFrame(np.zeros((t+1)*(t+1)).reshape(t+1,t+1))
df_forward_contract.index = [idx for idx in range(t, -1, -1)]

# Initialize t=10 as 106 (Face value 100 + coupon 6)
df_forward_contract[t] = face_value + coupon

# Backward (t=9 ~ t=1) (Discount Face value + coupon 6)
for col in range(t-1, -1, -1):
    for row in range(0, col+1):
        short_rate = df_short_rate.loc[row][col]
        upper_node = df_forward_contract.loc[row+1][col+1]
        lower_node = df_forward_contract.loc[row][col+1]
        df_forward_contract.loc[row][col]  = coupon + ( (1/2) * upper_node + (1/2) * lower_node) / (1 + short_rate)

# Backward (t=0) (Discount Face value)
df_forward_contract.loc[0][0]  = ((1/2) * df_forward_contract.loc[0][1] + (1/2) * df_forward_contract.loc[1][1]) / (1 + short_rate)

print(f"The price of a forward contract: {round(df_forward_contract.loc[0][0], 2)}")
print(f"Forward Contract: ")
df_forward_contract

The price of a forward contract: 91.72
Forward Contract: 


,0,1,2,3,4,5,6,7,8,9,10
10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,106.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,86.941560,106.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,77.132277,92.025686,106.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,72.675509,84.887989,96.278643,106.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,71.582527,81.988951,91.595308,99.754953,106.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,72.774990,81.906357,90.222727,97.220652,102.543101,106.0
4,0.000000,0.000000,0.000000,0.000000,75.632201,83.816132,91.152736,97.245612,101.822393,104.745522,106.0
3,0.000000,0.000000,0.000000,79.773375,87.212196,93.764405,99.117125,103.064729,105.512614,106.464430,106.0
2,0.000000,0.000000,84.947553,91.763869,97.653539,102.371235,105.765345,107.776765,108.425878,107.793403,106.0
1,0.000000,90.975690,97.240326,102.541528,106.690122,109.579055,111.178748,111.524139,110.697899,108.813437,106.0


- (b) Suppose this bond can be called by the issuing party at any time after 5 years. (When the bond is called, the face value plus the currently due coupon are paid at that time and the bond is canceled.) What is the fair value of this bond?          

In [30]:
# Initialize Bond price
df_forward_contract = pd.DataFrame(np.zeros((t+1)*(t+1)).reshape(t+1,t+1))
df_forward_contract.index = [idx for idx in range(t, -1, -1)]

# Initialize t=10 as 106 (Face value 100 + coupon 6)
df_forward_contract[t] = face_value + coupon

# Backward (t=9 ~ t=5) (Discount Face value + coupon 6) (able to early exercise)
for col in range(t-1, 5-1, -1):
    for row in range(0, col+1):
        short_rate = df_short_rate.loc[row][col]
        upper_node = df_forward_contract.loc[row+1][col+1]
        lower_node = df_forward_contract.loc[row][col+1]
        df_forward_contract.loc[row][col]  = min(face_value + coupon, \
                                                coupon + ( (1/2) * upper_node + (1/2) * lower_node) / (1 + short_rate))

# Backward (t=5 ~ t=1) (Discount Face value + coupon 6) (not able to early exercise)
for col in range(4, -1, -1):
    for row in range(0, col+1):
        short_rate = df_short_rate.loc[row][col]
        upper_node = df_forward_contract.loc[row+1][col+1]
        lower_node = df_forward_contract.loc[row][col+1]
        df_forward_contract.loc[row][col]  = coupon + ( (1/2) * upper_node + (1/2) * lower_node) / (1 + short_rate)

# Backward (t=0) (Discount Face value)
df_forward_contract.loc[0][0]  = ((1/2) * df_forward_contract.loc[0][1] + (1/2) * df_forward_contract.loc[1][1]) / (1 + short_rate)

print(f"The price of a forward contract: {round(df_forward_contract.loc[0][0], 2)}")
print(f"Forward Contract: ")
df_forward_contract

The price of a forward contract: 90.95
Forward Contract: 


,0,1,2,3,4,5,6,7,8,9,10
10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,106.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,86.941560,106.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,77.132277,92.025686,106.0
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,72.675509,84.887989,96.278643,106.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,71.582527,81.988951,91.595308,99.754953,106.0
5,0.000000,0.000000,0.000000,0.000000,0.000000,72.774990,81.906357,90.222727,97.220652,102.543101,106.0
4,0.000000,0.000000,0.000000,0.000000,75.632201,83.816132,91.152736,97.245612,101.822393,104.745522,106.0
3,0.000000,0.000000,0.000000,79.768824,87.202151,93.742440,99.069504,102.962288,105.293796,106.000000,106.0
2,0.000000,0.000000,84.897156,91.658917,97.437358,101.930581,104.876148,106.000000,106.000000,106.000000,106.0
1,0.000000,90.709233,96.719439,101.537199,104.780500,106.000000,106.000000,106.000000,106.000000,106.000000,106.0


- (c) Use the forward equation to find the spot rate curve for the lattice.

In [31]:
def find_elementary_price(T, P_0, df_short_rate):
    df_elementary_price = pd.DataFrame(np.zeros((T)*(T)).reshape(T, T))
    df_elementary_price.index = [idx for idx in range(T-1, -1, -1)]
    df_elementary_price[0][0] = P_0 
    
    for col in range(1, T):
        for row in range(0, col+1):
            # (1) j == 0
            if row == 0:
                df_elementary_price.loc[row][col] = (1/2) * (df_elementary_price.loc[row][col-1]) / (1+df_short_rate.loc[row][col-1])
         
            # (2) i == j 
            elif row == col:
                df_elementary_price.loc[row][col] = (1/2) * (df_elementary_price.loc[row-1][col-1]) / (1+df_short_rate.loc[row-1][col-1])
            
            # (3) i > j
            elif row < col:
                df_elementary_price.loc[row][col] = (1/2) * (df_elementary_price.loc[row-1][col-1]) / (1+df_short_rate.loc[row-1][col-1]) \
                                                    + (1/2) * (df_elementary_price.loc[row][col-1]) / (1+df_short_rate.loc[row][col-1])
            
    return df_elementary_price

df_elementary_price = find_elementary_price(t, 1, df_short_rate)
df_elementary_price

,0,1,2,3,4,5,6,7,8,9
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000615
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001549,0.006476
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003763,0.013990,0.029562
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.008874,0.028895,0.054281,0.077134
5,0.0,0.000000,0.000000,0.000000,0.000000,0.020398,0.057019,0.093799,0.118498,0.127204
4,0.0,0.000000,0.000000,0.000000,0.045872,0.107071,0.151173,0.167222,0.159586,0.137872
3,0.0,0.000000,0.000000,0.101256,0.189521,0.223361,0.212008,0.177139,0.136049,0.098435
2,0.0,0.000000,0.220008,0.309640,0.292509,0.231688,0.166077,0.111663,0.071822,0.044725
1,0.0,0.471698,0.443774,0.315081,0.199972,0.119588,0.068971,0.038832,0.021497,0.011754
0,1.0,0.471698,0.223766,0.106697,0.051113,0.024589,0.011874,0.005753,0.002796,0.001363


In [32]:
zcb_price = [0] * (t)
for col in range(1, t):
    zcb_price[col] = sum(df_elementary_price.loc[:, col])
df_zcb_price = pd.DataFrame({"ZCB Prices" : zcb_price}).T

spot_rate = [0] * (t)
for col in range(1, t):
    spot_rate[col] = ( (1 / df_zcb_price.iloc[0][col]) ** (1 / col) - 1 ) * 100 # (%)
df_spot_rate= pd.DataFrame({"Spot Rates" : spot_rate}).T

print(f"Spot Rate Curve: ")
df_spot_rate

Spot Rate Curve: 


,0,1,2,3,4,5,6,7,8,9
Spot Rates,0.0,6.0,6.146089,6.293906,6.443079,6.593199,6.743825,6.894485,7.044691,7.19394
